In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import lightning.pytorch as pl
import optuna
from optuna.pruners import HyperbandPruner
from optuna.integration import PyTorchLightningPruningCallback
torch.set_float32_matmul_precision('high')

In [2]:
from BPE_Encoder import token2id, train_dataset, val_dataset
from Transformer import MLLM

In [ ]:
# Train tokens
train_ids = torch.load("data\train_texttokens.pt", map_location="cpu")
train_ids_stream       = train_ids.tolist() 
print(len(train_ids_stream))

# validation set tokens 
val_ids = torch.load("data\val_texttokens.pt", map_location="cpu")
val_ids_stream       = val_ids.tolist()
print(len(val_ids_stream))

2851449
480708


In [4]:

def objective(trial):


    seed = 2025 + trial.number
    pl.seed_everything(seed, workers=True)


    lr   = trial.suggest_float("lr", 1e-4, 1e-3, log=True)
    wd   = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)
    dropout_percentage = trial.suggest_float("dropout_percentage", 0, 0.15)

    heads      = trial.suggest_categorical("heads", [2,4])
    num_layers = trial.suggest_categorical("num_layers", [2,3,4,6])
    dim        = trial.suggest_categorical("dim", [48,64])

    ls         = trial.suggest_float("label_smoothing", 0, 0.06)
    pct_start = trial.suggest_float("pct_start", 0.1, 0.5)
    act = trial.suggest_categorical("activation", ["relu", "gelu"])
    ffn_internal = trial.suggest_categorical('ffn_internal', [2,4])


    model = MLLM(vocab=len(token2id), dim=dim, pad_idx=token2id['<pad>'], max_pos=256, QKV_dim=dim // heads, heads=heads, num_layers=num_layers, 
                 dropout_percentage=dropout_percentage, learning_rate=lr, wd=wd, ls=ls, pct_start = pct_start, act=act, ffn_internal = ffn_internal)

    callback_prune = PyTorchLightningPruningCallback(trial, monitor="Val_Loss")

    early_call= pl.callbacks.EarlyStopping( monitor="Val_Loss", patience=5, mode="min")


    trainer = pl.Trainer(
        max_epochs=30,
        accelerator="gpu",
        precision="16-mixed",
        gradient_clip_val=1.0,        
        callbacks=[callback_prune,early_call],
        logger=False,
        enable_checkpointing=False,
    )

    trainer.fit(model, train_dataset, val_dataset)

    val_loss = trainer.callback_metrics["Val_Loss"].item()


    return val_loss



pruner = HyperbandPruner(min_resource=1, max_resource=30, reduction_factor=3)

study = optuna.create_study(direction="minimize", pruner=pruner)
study.optimize(objective, n_trials=25) 

print("Best trial:", study.best_trial.params)


[I 2025-07-11 19:35:21,879] A new study created in memory with name: no-name-6e9c7620-1c8f-4734-a3bc-2e48a5edd92e
Seed set to 2025
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name       | Type             | Params | Mode 
--------------------------------------------------------
0 | act        | ReLU             | 0      | train
1 | embedtoken | Embedding        | 1.3 M  | train
2 | embedpos   | Embedding        | 16.4 K | train
3 | dropout    | Dropout          | 0      | train
4 | layers     | ModuleList       | 200 K  | train
5 | model_head | Linear           | 1.3 M  | train
6 | loss       | CrossEntropyLoss | 0      | train
--------------------------------------------------------
1.5 M     Trainable params
0         Non-trainable params
1.5 M     Total par

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

[I 2025-07-11 22:59:27,055] Trial 24 pruned. Trial was pruned at epoch 1.


Best trial: {'lr': 0.0008846917512346465, 'weight_decay': 0.00029475907910799643, 'dropout_percentage': 0.0010306170289606452, 'heads': 4, 'num_layers': 6, 'dim': 64, 'label_smoothing': 0.019019051540349757, 'pct_start': 0.35980068988151803, 'activation': 'gelu', 'ffn_internal': 4}
